# Econ 8210 Quant Macro, Homework 1
## Part 2 - Solution Methods
Haosi Shen, Fall 2024

In [1]:
# Housekeeping
import numpy as np
import pandas as pd
import time
import torch

np.random.seed(42) 

## Computing Pareto Efficient Allocations

Consider an endowment economy with $m$ different goods and $n$ agents. Each agent $i = 1, ..., n$ has an endowment $e_j^i >0$ for every $j = 1, ..., m$ and a utility function of the form
$$ u^i (x) = \sum_{j=1}^{m} \alpha_j \frac{x_{j}^{1+\omega_j^i}}{1+\omega_j^i} $$
where $\alpha_j > 0 > \omega_j^i$ are agent-specific parameters.

Given some social weights $\lambda_i > 0$, solve for the social planner’s problem for $m = n = 3$ using the **Adam (Adaptive Moment Estimation)** method. Try different values of $\alpha_j,\;\omega_j^i,\;\lambda_i$. 

$$ \max_{\{x^i\}} \; \sum_{i=1}^{n} \lambda_i u^{i}(x)$$

Compute first the case where all the agents have the same parameters and
social weights and later a case where there is a fair degree of heterogeneity.

How does the method perform? How does heterogeneity in the agent-specific parameters
affect the results?

Can you handle the case where $m = n = 10$?

> I choose to use **Adam** since it is more efficient for high-dimensional optimization problems and offers more stability and robustness. However, if we are only solving for the case of $m=n=3$, then the Newton-Raphson method might be more ideal since this problem is relatively smooth. Adam only requires gradient information and does not involve inverting the Hessian.

In [2]:
# utility function for agent i
def utility(x, alpha, omega):
    return torch.sum(alpha * (x ** (1 + omega)) / (1 + omega))


# Incorporate resource constraints for each good j
# Define total endowments
endowments = torch.tensor([30.0, 30.0, 30.0], dtype = torch.float32)
# endowments = torch.tensor([10.0, 20.0, 40.0], dtype = torch.float32)

### Case I: Homogeneous Agents

> $m = n = 3$

All agents $j$ have the same parameters $\alpha_j, \omega_j^i$ and social weights $\lambda_j$. 

In [3]:
# Social planner's objective, with Homogeneous Agents
def social_planner_objective_homog(x, alpha_j, omega_j, lambda_i, endowments, penalty_weight = 1000):
    total_utility = 0
    total_allocations = torch.zeros(m)

    for i in range(n):
        x_i_softplus = torch.nn.functional.softplus(x[i]) # ensure nonneg allocations
        total_utility += lambda_i[i] * utility(x_i_softplus, alpha_j, omega_j)
        total_allocations += x_i_softplus

    # penalize if RC is violated, i.e. total allocations > endowments
    penalty = torch.sum(torch.clamp(total_allocations - endowments, min = 0) ** 2)

    return -total_utility + penalty_weight * penalty   # maximization, so take negative

In [4]:
import torch.optim as optim

# =============== DEFINE PARAMETERS ===============
m, n = 3, 3  # 3 goods, 3 agents
alpha_j = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32) 
omega_j = torch.tensor([[0.5, 0.5, 0.5]] * n, dtype = torch.float32)
lambda_i = torch.tensor([1.0, 1.0, 1.0], dtype = torch.float32)  # Pareto weights

# initial allocations (using small positive values)
x_i = torch.rand((n, m), requires_grad = True)

# Set up optimizer
optimizer = optim.Adam([x_i], lr = 0.01)

# Optimization
num_iterations = 1000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_homog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 100 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_homog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Homogeneous Agents:")
print(df_final_homog)


Iteration 99: Objective = 47.90167236328125
Iteration 199: Objective = 97.20240020751953
Iteration 299: Objective = 165.30142211914062
Iteration 399: Objective = 246.75
Iteration 499: Objective = 338.384033203125
Iteration 599: Objective = 438.5430908203125
Iteration 699: Objective = 546.2037353515625
Iteration 799: Objective = 569.2835693359375
Iteration 899: Objective = 569.2864379882812
Iteration 999: Objective = 569.28759765625

 Optimal allocations for Homogeneous Agents:
            Good 1     Good 2     Good 3
Agent 1   9.939378   9.994959   9.863128
Agent 2  10.019382   9.975774  10.016639
Agent 3  10.045987  10.034011  10.124976


### Case II: Heterogeneous Agents 
> $m = n = 3$

* Each agent $i$ has their own set of $\alpha_j, \omega_j^i$ parameters.
* Pareto weights $\lambda_i$ differ among agents.
* Resource constraint remains the same. Total endowment for each good is 30.

In [5]:
# Social planner's objective, with Heterogeneous Agents

def social_planner_objective_heterog(x, alpha_j, omega_j, lambda_i, endowments, penalty_weight = 1000):
    total_utility = 0
    total_allocations = torch.zeros(m)

    for i in range(n):
        x_i_softplus = torch.nn.functional.softplus(x[i]) # ensure nonneg allocations
        total_utility += lambda_i[i] * utility(x_i_softplus, alpha_j[i], omega_j[i])
        total_allocations += x_i_softplus

    # penalize if RC is violated, i.e. total allocations > endowments
    penalty = torch.sum(torch.clamp(total_allocations - endowments, min = 0) ** 2)

    return -total_utility + penalty_weight * penalty   # maximization, so take negative


In [6]:
# =============== DEFINE PARAMETERS ===============
alpha_j = torch.tensor([[1.0, 0.8, 1.2],  # agent 1
                        [1.1, 0.9, 1.3],  # agent 2
                        [0.9, 0.7, 1.1]])  # agent 3

omega_j = torch.tensor([[0.3, 0.5, 0.7], 
                        [0.4, 0.6, 0.8],  
                        [0.5, 0.4, 0.6]]) 

lambda_i = torch.tensor([0.9, 1.1, 1.0])  # Social weights


# initial allocations (using small positive values)
x_i = torch.rand((n, m), requires_grad = True)
#x_i = (torch.rand((n, m), requires_grad=True) * endowments / n).clone().detach().requires_grad_(True)

# Set up optimizer
optimizer = optim.Adam([x_i], lr = 0.001)

# Optimization
num_iterations = 10000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_heterog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 1000 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_heterog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Heterogeneous Agents:")
print(df_final_heterog)

Iteration 999: Objective = 15.176987648010254
Iteration 1999: Objective = 32.06589126586914
Iteration 2999: Objective = 54.335941314697266
Iteration 3999: Objective = 80.09893798828125
Iteration 4999: Objective = 108.60932922363281
Iteration 5999: Objective = 139.5780487060547
Iteration 6999: Objective = 172.86007690429688
Iteration 7999: Objective = 208.35690307617188
Iteration 8999: Objective = 221.42538452148438
Iteration 9999: Objective = 227.85813903808594

 Optimal allocations for Heterogeneous Agents:
            Good 1     Good 2     Good 3
Agent 1   9.087026   9.912905   9.912929
Agent 2  10.720445  11.217888  11.453905
Agent 3  10.193778   8.870395   8.635993


> The `Adam` optimizer performs fairly well for both the homogeneous and heterogeneous agents cases. The computations are fast and produce sufficiently accurate results. 
> 
> Introducing heterogeneity in the agent-specific parameters does affect the socially optimal allocations, and significantly slow down convergence of optimization.

### Case III: 10 Agents, 10 Goods

> $m=n=10$

Since the `Adam` optimizer generally works well for higher-dimension problems, we now try computing the Pareto efficient allocations of a 10-agent 10-good economy. 

**Consider homogeneity across agents.**

In [7]:
m, n = 10, 10  # 10 goods, 10 agents
endowments = torch.tensor([30.0] * m)  # total endowments for 10 goods

alpha_j = torch.tensor([1.0] * m)  
omega_j = torch.tensor([0.5] * m)  
lambda_i = torch.tensor([1.0] * n)

x_i = torch.rand((n, m), requires_grad = True)  # initial allocations

optimizer = optim.Adam([x_i], lr = 0.01)

# Optimization
num_iterations = 1000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_homog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 100 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_homog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Homogeneous Agents:")
print(df_final_homog)

Iteration 99: Objective = 162.29444885253906
Iteration 199: Objective = 338.72503662109375
Iteration 299: Objective = 346.544677734375
Iteration 399: Objective = 346.9432067871094
Iteration 499: Objective = 347.1405944824219
Iteration 599: Objective = 346.41668701171875
Iteration 699: Objective = 347.0489807128906
Iteration 799: Objective = 347.0523681640625
Iteration 899: Objective = 346.85882568359375
Iteration 999: Objective = 347.0088195800781

 Optimal allocations for Homogeneous Agents:
            Good 1    Good 2    Good 3    Good 4    Good 5    Good 6  \
Agent 1   3.332024  3.467447  3.013774  2.921800  3.370348  2.606249   
Agent 2   2.523771  3.006770  2.936013  3.410774  3.116770  2.957279   
Agent 3   3.396104  3.225600  3.357541  2.552707  2.610854  3.233714   
Agent 4   3.280476  2.671251  3.462526  2.876756  3.260174  2.962018   
Agent 5   2.772020  3.470001  3.426079  3.285574  3.149989  2.696598   
Agent 6   3.035664  2.449300  2.839202  2.810548  3.096043  2.705003  

**Now, try the heterogeneous agents case.**

In [8]:
# Heterogeneous agent-specific parameters
alpha_j = torch.tensor([
    [1.0, 0.8, 1.2, 0.9, 1.1, 0.7, 1.3, 0.95, 1.05, 1.15] for _ in range(n)]) 

omega_j = torch.tensor([
    [0.3, 0.4, 0.5, 0.35, 0.45, 0.55, 0.25, 0.6, 0.65, 0.5] for _ in range(n)])

lambda_i = torch.tensor([0.9, 1.1, 0.8, 1.2, 1.0, 0.85, 1.15, 0.95, 1.05, 1.0])


In [9]:
x_i = torch.rand((n, m), requires_grad=True)
optimizer = optim.Adam([x_i], lr = 0.001)

# Optimization
num_iterations = 10000
for iteration in range(num_iterations):
    optimizer.zero_grad()
    objective = social_planner_objective_heterog(x_i, alpha_j, omega_j, lambda_i, endowments)
    objective.backward()
    optimizer.step()

    if (iteration+1) % 1000 == 0:
        print(f"Iteration {iteration}: Objective = {-objective.item()}")


# Final optimal allocations
final_allocations = torch.nn.functional.softplus(x_i).detach().numpy()

df_final_heterog = pd.DataFrame(
    final_allocations,
    index = [f'Agent {i+1}' for i in range(final_allocations.shape[0])],
    columns = [f'Good {j+1}' for j in range(final_allocations.shape[1])]
)

print("\n Optimal allocations for Heterogeneous Agents:")
print(df_final_heterog)

Iteration 999: Objective = 163.5595245361328
Iteration 1999: Objective = 327.0325622558594
Iteration 2999: Objective = 350.0217590332031
Iteration 3999: Objective = 359.7204284667969
Iteration 4999: Objective = 383.3825378417969
Iteration 5999: Objective = 425.12237548828125
Iteration 6999: Objective = 479.7190246582031
Iteration 7999: Objective = 534.1912841796875
Iteration 8999: Objective = 585.0552978515625
Iteration 9999: Objective = 626.8079833984375

 Optimal allocations for Heterogeneous Agents:
            Good 1    Good 2     Good 3    Good 4    Good 5    Good 6  \
Agent 1   0.125988  0.150368   0.046247  0.242932  0.060060  0.066638   
Agent 2   6.712726  9.160335   9.289027  6.410796  7.141737  8.688856   
Agent 3   0.107602  0.083613   0.053757  0.078370  0.064677  0.084108   
Agent 4   8.337194  9.270158  10.074583  8.461432  9.624029  9.113145   
Agent 5   1.219632  0.271760   0.103095  0.242591  0.085858  0.106013   
Agent 6   0.129397  0.090682   0.056293  0.137855  0.0

> The method works well for $m=n=10$, even after we introduce household heterogeneity. 

## Computing Equilibrium Allocations

Using the same model as in the previous exercise. Find the equilibrium prices for each good $p^j$.

1. Solve for the first-order conditions of each agent.
1. Aggregate the excess demands. 
1. Solve the resulting system of nonlinear equations, when all markets clear.

### Homogeneous Agents, $m=n=3$

**FOCs**
> To find the demand for each good, we solve the Lagrangean:
> $$\mathcal{L}=\sum_{j=1}^{m}\alpha_j \cdot \frac{x_{ij}^{1+\omega}}{1+\omega}-\lambda_i \Big(\sum_{j=1}^{m}p_j\cdot x_{ij} - Y_i\Big)$$
> where $Y_i$ is the income of agent $i$. 
> The FOCs for this problem yield
> $$ x_{ij} = \frac{\alpha_j \cdot p_{j}^{-\omega}}{\sum_{k=1}^{m} \alpha_k \cdot p_{k}^{-\omega} }\cdot Y_i $$

**Excess Demand**
> Aggregate the demand across agents and compare it to the total endowment to find the excess demand for each good
> $$z_j(p) =  \sum_{i=1}^{n} x_{ij}(p) - e_j$$
> Markets clear when $z_j(p)=0$ for all goods $j$.

**Equilibrium Prices**
> The goal is to find prices $p$ such that the excess demand is zero for all goods. This involves numerically solving a system of nonlinear equations.

In [10]:
from scipy.optimize import fsolve

# Step 1: Solve for individual agent's demand from FOC

def agent_demand(p, alpha, omega, income):
    """
    - p: prices for each good
    - alpha: preference weights
    - omega: curvature parameter
    - income: agent's income

    Returns: demand for each good
    """
    return (alpha * (p ** -omega)) / (np.sum(alpha * (p ** -omega))) * income



# Step 2: Solve for excess demand for each good

def excess_demand(p, endowments, alpha, omega, incomes):
    """
    - endowments: total endowment for each good 
    - incomes: total income for each agent

    Returns: excess demand for each good 
    """
    n, m = incomes.shape[0], endowments.shape[0]
    total_demand = np.zeros(m)
    
    for i in range(n):
        demand_i = agent_demand(p, alpha, omega, incomes[i])
        total_demand += demand_i
    
    return total_demand - endowments 


In [11]:
# Step 3: Solve for equilibrium prices where excess demand = 0
def find_eq_prices(endowments, alpha, omega, incomes):
    """
    Solve for eq prices using `fsolve` to find root of excess demand function.
    """
    p0 = np.ones(endowments.shape[0])   # Initial guess
    
    eq_prices = fsolve(lambda p: excess_demand(p, endowments, alpha, omega, incomes), p0)
    
    return eq_prices


In [12]:
# =============== DEFINE PARAMETERS (homogeneity) ===============
m, n = 3, 3  # 3 goods, 3 agents
alpha_j = np.ones(m)  # preference weights
omega_j = 0.5  # curvature param
endowments = np.array([30.0, 30.0, 30.0])  # total endowments
incomes = np.ones(n) * np.sum(endowments) / n  # equally distributed income

eq_prices_homog = find_eq_prices(endowments, alpha_j, omega_j, incomes)
print("Equilibrium Prices for Homogeneous Agents:", eq_prices_homog)

Equilibrium Prices for Homogeneous Agents: [1. 1. 1.]


By the FWT, every competitive equilibrium is Pareto efficient. Equilibrium prices align with marginal rates of substitution, and budget constraints enforce resource constraints. Therefore, the Arrow-Debreu equilibrium allocations coincide with the Pareto-efficient allocations.

### Heterogeneous Agents, $m=n=3$


**FOCs**
> To find the demand for each good, we solve the Lagrangean:
> $$\mathcal{L}=\sum_{j=1}^{m}\alpha_j^i \cdot \frac{x_{ij}^{1+\omega_j^i}}{1+\omega_j^i}-\lambda_i \Big(\sum_{j=1}^{m}p_j\cdot x_{ij} - Y_i\Big)$$
> where $Y_i$ is the income of agent $i$. 
> The FOCs for this problem yield
> $$ x_{ij} = \frac{\alpha_j^i \cdot p_{j}^{-\omega_j^i}}{\sum_{k=1}^{m} \alpha_k^i \cdot p_{k}^{-\omega_j^i} }\cdot Y_i $$

**Excess Demand**
> Aggregate the demand across agents and compare it to the total endowment to find the excess demand for each good
> $$z_j(p) =  \sum_{i=1}^{n} x_{ij}(p) - e_j$$
> Markets clear when $z_j(p)=0$ for all goods $j$.

**Equilibrium Prices**
> The goal is to find prices $p$ such that the excess demand is zero for all goods. This involves numerically solving a system of nonlinear equations.

In [13]:
# Step 1: Individual agent's demand
def agent_demand_heterog(p, alpha, omega, income):
    demand = (alpha * (p ** -omega)) / (np.sum(alpha * (p ** -omega))) * income
    return demand

# Step 2: excess demand function
def excess_demand_heterog(p, endowments, alpha, omega, incomes):
    n, m = incomes.shape[0], endowments.shape[0]
    total_demand = np.zeros(m)
    
    # Aggregate demand across all agents
    for i in range(n):
        demand_i = agent_demand_heterog(p, alpha[i], omega[i], incomes[i])
        total_demand += demand_i
    
    excess_demand = total_demand - endowments
    return excess_demand


# Step 3: Solve for equilibrium prices where excess demand = 0
def find_eq_prices_heterog(endowments, alpha, omega, incomes):
    # Initial guess for prices
    p0 = np.ones(endowments.shape[0])
    
    eq_prices = fsolve(lambda p: excess_demand_heterog(p, endowments, alpha, omega, incomes), p0)
    
    return eq_prices

In [14]:
# =============== DEFINE PARAMETERS (homogeneity) ===============
m, n = 3, 3  # 3 goods, 3 agents
alpha_j = np.array([[1.0, 0.8, 1.2],  # agent 1
                    [1.1, 0.9, 1.3],  # agent 2
                    [0.9, 0.7, 1.1]])  # agent 3

omega_j = np.array([[0.3, 0.5, 0.7], 
                    [0.4, 0.6, 0.8],  
                    [0.5, 0.4, 0.6]]) 


endowments = np.array([30.0, 30.0, 30.0])  # total endowments
incomes = np.ones(n) * np.sum(endowments) / n  # equally distributed income

eq_prices_heterog = find_eq_prices_heterog(endowments, alpha_j, omega_j, incomes)

print("Equilibrium Prices for Heterogeneous Agents:", eq_prices_heterog)

Equilibrium Prices for Heterogeneous Agents: [1.2410273  0.75897273 1.47233807]
